<a href="https://colab.research.google.com/github/AngelMasterr/Machine-Learning-with-Python/blob/main/24_10_2024_Fundamentos_del_RAG_%7C_EvoAcademy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![picture](https://drive.google.com/uc?id=1t85VSkuEnCm-X8egDjib0GMTGZT0LM3c)

# Fundamentos del Retrieval-Augmented Generation (RAG)
Preparado por [Sebastián Cisterna](https://www.linkedin.com/in/scisterna/)

# ¿Qué es el RAG?
RAG o también Retrieval-augmented Generation es una técnica que combina la recuperación de información (retrieval) con la generación de texto.

Este enfoque permite a los modelos de lenguaje aprovechar una base de datos externa de textos para complementar y enriquecer sus respuestas, mejorando la relevancia y exactitud de la información generada.

Lo que veremos en este tutorial se resume en el siguiente flujo

![picture](https://drive.google.com/uc?id=1ACV-UtacZStN5ItO0i0-mwh6N7kxqFMs)

## Preparando el ambiente
Instalamos las librerías y configuramos nuestra API Key

In [1]:
!pip install -U -q langchain langchain-openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.6/50.6 kB 519.5 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 407.7/407.7 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.9/296.9 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 386.9/386.9 kB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.2/325.2 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.5/144.5 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/5

In [2]:
import numpy as np
import pandas as pd
import os

from langchain_core.output_parsers import StrOutputParser

# Configuracion splitter
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Configuracion LLM - GPT
from langchain_openai import ChatOpenAI

# Para obtener la API desde Secrets en Google Colab. Si usas otro entorno debes adaptar esto
from google.colab import userdata
OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

llm = ChatOpenAI(model="gpt-4o-mini")

SecretNotFoundError: Secret OPENAI_API_KEY does not exist.

# Prueba sin RAG
Llamamos el LLM sin ninguna modificación y vemos que alucina o bien desconoce esta información

In [3]:
llm.invoke("¿Donde se va a transmitir WWE RAW en 2025?").content

NameError: name 'llm' is not defined

# Configurando el RAG

## 0. Importando los documentos
Normalmente esto sería más complejo, pero para entender los fundamentos vamos a implementar como un texto directo en la plataforma.

In [4]:
# fuente: https://www.infobae.com/que-puedo-ver/2024/01/23/wwe-en-netflix-raw-y-mas-eventos-llegaran-desde-2025/
noticia_1 = """
Netflix es la nueva casa de la WWE: “Raw” y más eventos especiales se podrán ver en la plataforma

El famoso programa de lucha libre llegará al streaming. WrestleMania, SummerSlam y Royal Rumble también se sumarán el próximo año.

Netflix ha anunciado un histórico acuerdo con la empresa de entretenimiento deportivo WWE. A partir de 2025, la “N” roja será el hogar exclusivo del popular programa Monday Night Raw en Estados Unidos, Canadá, Reino Unido y Latinoamérica.
Este acuerdo, que tiene planes de expandirse a más territorios alrededor de 190 países, representa un cambio trascendental para Raw, ya que se traslada de la televisión tradicional a la plataforma de streaming por primera vez desde su lanzamiento en 1993.

El acuerdo, con una duración de diez años y un valor que supera los 5 mil millones de dólares, también incluirá todos los programas y especiales de la WWE, como SmackDown y NXT, además de los eventos premium en vivo del calibre de WrestleMania, SummerSlam y Royal Rumble.
El gigante del streaming también ofrecerá documentales galardonados de la marca, series originales y proyectos futuros a su audiencia internacional a partir del próximo año.

El impacto de la WWE
Actualmente, Raw es el programa número uno en la cadena USA Network, atrayendo a 17.5 millones de espectadores únicos a lo largo del año, y se mantiene como uno de los programas más exitosos en el demográfico publicitario de 18 a 49 años.
Adicionalmente, WWE cuenta con más de mil millones de seguidores en sus plataformas de redes sociales.

El programa Monday Night Raw es uno de los más icónicos del entretenimiento deportivo, acumulando 1600 episodios desde su debut. Combinando lo mejor del contenido guionizado con el entretenimiento en vivo, el show semanal ha impulsado las carreras de múltiples celebridades en el mundo de la lucha libre con el paso del tiempo, al punto de convertirse en un referente mundial.
"""

# fuente: https://www.emol.com/noticias/Economia/2023/09/28/1108504/luksin-renuncia-presidencia-quinenco-ccu.html
noticia_2 = """
Andrónico Luksic renuncia a la presidencia de Quiñenco y a directorios de otras empresas.

El empresario Andrónico Luksic anunció su renuncia a la presidencia de Quiñenco y de los directorios de otras compañías de las que forma parte. La renuncia se hará efectiva desde el 29 de septiembre. También dejará presidencias de Compañía Cervecerías Unidas (CCU) y LQ Inversiones Financieras (LQIF), las vicepresidencias de Banco de Chile y Compañía Sud Americana de Vapores (CSAV), y el directorio de Invexans.

"Tras un profundo proceso de reflexión he llegado a la convicción de que es momento de alejarme del día a día, de dar paso a otros liderazgos y de permitir que sea el gran equipo de profesionales que hemos construido a lo largo de los años, el que conduzca a nuestras empresas hacia el futuro", sostuvo el empresario.

En Quiñenco, Luksic recalcó haber podido "profundizar la estrategia de diversificación internacional que nos trazamos, al punto que en 2022 más de 90% de la utilidad que obtuvimos provino del extranjero.

También ha sido fundamental la creación, desde 2014, de áreas como relaciones laborales, desarrollo organizacional, asuntos corporativos y sustentabilidad, que incorporaron a nuestra matriz industrial y financiera variables propias de las nuevas exigencias que la sociedad y el mundo demandan de las empresas".

Por su parte, Francisco Pérez Mackenna, quien asumirá la presidencia de Quiñenco, señaló que "la decisión que conocimos hoy es totalmente personal, y fue adoptada por él en forma muy responsable y reflexiva, buscando siempre un mejor futuro para las compañías".

De acuerdo a la determinación adoptada en cada uno de los respectivos directorios, los cambios a concretarse a contar del 29 de diciembre próximo son:
- En el directorio de Quiñenco, asumirá como presidente Pablo Granifo Lavín, en tanto que Paola Luksic Fontbona, actual asesora, se incorporará como directora.
- En Banco de Chile, Francisco Pérez Mackenna será el nuevo vicepresidente, y se sumará en calidad de director Patricio Jottar Nasrallah.
- En CCU, el presidente será Francisco Pérez, y Óscar Hasbún Martínez se incorporará como director.
- En LQIF, Francisco Pérez será el presidente, y el cargo de director será asumido por Rodrigo Hinzpeter Kirberg.
- En Invexans, Vicente Mobarec Katunaric será el nuevo integrante del directorio.
- Y en CSAV, Pablo Granifo se incorpora como director y vicepresidente de la compañía.
"""

noticia_3 = """
Siendo su segundo encuentro como inicialista, se esperaba que el cafetero sumara más minutos en el campo de juego,
aprovechando su buen momento y la cantidad de comentarios en el que pedían que se le diera más oportunidades para demostrar su calidad.

Sin embargo, ocurrió todo lo contrario en el partido de Aston Villa y Bologna,
debido a que el técnico Unai Emery decidió sacar a Jhon Jader Durán de la cancha a los 65 minutos,
para darle espacio a Ollie Watkins, que viene siendo inicialista habitual con los británicos.

Todo se presentó a los 65 minutos, en el momento justo después de la anotación del colombiano,
que luego de terminar de celebrar junto a sus compañeros el 2-0, la paleta de cambios mostró el número 9 del cafetero,
lo que no cayó nada bien en el deportista y aficionados.
"""

In [5]:
from langchain.docstore.document import Document

doc_1 =  Document(noticia_1)
doc_2 =  Document(noticia_2)
doc_3 =  Document(noticia_3)
docs =  [doc_1, doc_2, doc_3]

## 1. Diviviendo los documentos en chunks
Los documentos son divididos en pedacitos, o chunks, para que puedan ser correctamente procesados en este contexto.

In [6]:
text_splitter = RecursiveCharacterTextSplitter(
                  chunk_size=450,
                  chunk_overlap=0)

splits = text_splitter.split_documents(docs)

for index, split in enumerate(splits):
  print(f"SPLIT {index + 1}")
  print(split.page_content)
  print("--")

SPLIT 1
Netflix es la nueva casa de la WWE: “Raw” y más eventos especiales se podrán ver en la plataforma

El famoso programa de lucha libre llegará al streaming. WrestleMania, SummerSlam y Royal Rumble también se sumarán el próximo año.
--
SPLIT 2
Netflix ha anunciado un histórico acuerdo con la empresa de entretenimiento deportivo WWE. A partir de 2025, la “N” roja será el hogar exclusivo del popular programa Monday Night Raw en Estados Unidos, Canadá, Reino Unido y Latinoamérica.
--
SPLIT 3
Este acuerdo, que tiene planes de expandirse a más territorios alrededor de 190 países, representa un cambio trascendental para Raw, ya que se traslada de la televisión tradicional a la plataforma de streaming por primera vez desde su lanzamiento en 1993.
--
SPLIT 4
El acuerdo, con una duración de diez años y un valor que supera los 5 mil millones de dólares, también incluirá todos los programas y especiales de la WWE, como SmackDown y NXT, además de los eventos premium en vivo del calibre de Wre

## 2. Guardando los chunks
Para los efectos de este ejercicio básico, almacenaremos los datos en un DataFrame. Una estructura muy similar a un Excel.

Durante el curso veremos que para aplicaciones en producción, esto se almacena en una base de datos vectorial.

In [7]:
# Creamos un dataFrame con los textos

chunks = []
for split in splits:
  chunks.append(split.page_content)

df = pd.DataFrame(chunks, columns = ['Text'])

# visualiza el dataframe
df

,Text
0,Netflix es la nueva casa de la WWE: “Raw” y má...
1,Netflix ha anunciado un histórico acuerdo con ...
2,"Este acuerdo, que tiene planes de expandirse a..."
3,"El acuerdo, con una duración de diez años y un..."
4,"El impacto de la WWE\nActualmente, Raw es el p..."
5,El programa Monday Night Raw es uno de los más...
6,Andrónico Luksic renuncia a la presidencia de ...
7,El empresario Andrónico Luksic anunció su renu...
8,"""Tras un profundo proceso de reflexión he lleg..."
9,"En Quiñenco, Luksic recalcó haber podido ""prof..."


## Generando los embeddings
Los embeddings son representaciones númericas de la información. Existen varios proveedores de Embeddings, pero uno de los más utilizados es el de OpenAI

In [ ]:
# Usaremos el modelo de Embeddings de OpenAI
from langchain_openai import OpenAIEmbeddings
embeddings_model = OpenAIEmbeddings()

# Crearemos una función que luego aplicaremos a cada texto,
def embeddings_fn(text):
  """
  Crea los embeddings dado un texto
  """
  return embeddings_model.embed_query(text)

### Remplaza el codigo de OpenAIEmbeddings por SentenceTransformers

In [ ]:
!pip install -U sentence_transformers

In [ ]:
from sentence_transformers import SentenceTransformer

# Carga un modelo pre-entrenado. Hay varios modelos disponibles, elige uno que se adapte a tus necesidades
# Puedes ver una lista de modelos aquí: https://www.sbert.net/docs/pretrained_models.html
embeddings_model = SentenceTransformer('all-mpnet-base-v2') # This line was incorrectly indented

In [12]:
def embeddings_fn(text):
      """
      Crea los embeddings dado un texto
      """
      return embeddings_model.encode(text)

In [17]:
# Calculamos los embeddings para cada texto en nuestro DataFrame
df['Embeddings'] = df['Text'].apply(embeddings_fn)

# Desplegamos el resultado, pueden ver que ahora tenemos una columna de Embeddings
df.head(10)

,Text,Embeddings
0,Netflix es la nueva casa de la WWE: “Raw” y má...,"[-0.06687324, 0.011862769, -0.0015801489, -0.0..."
1,Netflix ha anunciado un histórico acuerdo con ...,"[-0.025815295, 0.06348699, 0.016179634, -0.029..."
2,"Este acuerdo, que tiene planes de expandirse a...","[-0.032133304, 0.0071289465, 0.001991683, -0.0..."
3,"El acuerdo, con una duración de diez años y un...","[-0.05403012, 0.015692342, 0.0041304426, 0.000..."
4,"El impacto de la WWE\nActualmente, Raw es el p...","[-0.06468614, 0.044428598, 0.031938437, -0.036..."
5,El programa Monday Night Raw es uno de los más...,"[-0.03583354, 0.017398827, 0.025049526, -0.006..."
6,Andrónico Luksic renuncia a la presidencia de ...,"[-0.00029733483, 0.041497916, 0.0049358127, 0...."
7,El empresario Andrónico Luksic anunció su renu...,"[0.029257253, 0.042337183, -0.0029147905, 0.04..."
8,"""Tras un profundo proceso de reflexión he lleg...","[0.014808676, 0.027258983, -0.0040117437, -0.0..."
9,"En Quiñenco, Luksic recalcó haber podido ""prof...","[0.018315954, 0.023917312, -0.009063801, -0.02..."


¿Cuántas dimensiones tiene cada vector?

In [18]:
len(df['Embeddings'][0])

768

## Buscando el chunk más relevante

In [19]:
def find_best_passage(query, dataframe):
  """
  Calcula las distancias entre la consulta y cada documento en el marco de datos utilizando el producto punto.
  """
  query_embedding = embeddings_model.embed_query(query)

  # Revisa como opera la función dot aquí: https://numpy.org/doc/stable/reference/generated/numpy.dot.html
  dot_products = np.dot(np.stack(dataframe['Embeddings']), query_embedding)

  # Obtiene la ID del que más se parece
  idx = np.argmax(dot_products)

  return dataframe.iloc[idx]['Text'] # Devuelve el texto del índice con el valor máximo.

Notese que nuestro ejercicio entrega sólo un resultado. Sin embargo, veremos ejemplos donde se entregan más resultados de vuelta, por ejemplo los mejores 4.

In [20]:
query = "¿ donde vive el pibe valderrama ?"
best_passage = find_best_passage(query, df)
best_passage

AttributeError: 'SentenceTransformer' object has no attribute 'embed_query'

## Generando la respuesta con un LLM

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()


prompt = ChatPromptTemplate.from_messages([
    ("system", """
      Eres un bot servicial e informativo que responde preguntas utilizando el texto referencia incluido a continuación.
      Asegúrate de responder en una oración completa, siendo exhaustivo y proporcionando toda la información de fondo relevante.
      Sin embargo, estás hablando con una audiencia no técnica, por lo que debes desglosar los conceptos complicados y mantener un tono amigable y conversacional.
      Por favor, responde en el idioma de la pregunta.

      PREGUNTA: '{query}'
      TEXTO DE REFERENCIA: '{relevant_passage}'

      RESPUESTA:
    """
    ),
])

chain = prompt | llm | output_parser

In [ ]:
chain.invoke({
    "query": query,
    "relevant_passage": best_passage,
    })

'No tengo información específica sobre dónde vive el pibe Valderrama, pero lo que sí sé es que es un destacado futbolista colombiano reconocido por su talento en el campo de juego. En su carrera, ha sido muy querido y ha recibido muchos comentarios positivos sobre su habilidad, lo que ha llevado a que la gente espere verlo jugar más. Si estás interesado en saber más sobre él o su carrera, ¡estaré encantado de ayudarte!'

# Documentación
* [Quickstart - Q&A with RAG](https://python.langchain.com/docs/use_cases/question_answering/quickstart/)

Articulo recomendado
* [Cómo personalizar ChatGPT y otros LLMs | RAG, Fine-tuning, y más](https://www.evoacademy.cl/como-personalizar-chatgpt-y-otros-llms-rag-fine-tuning-y-mas/)

Podcast recomendado
* [Qué es el RAG](https://www.youtube.com/watch?v=hrATOtHZqEE).


---
# Síguenos:
- Spotify: [Escucha nuestro podcast sobre IA](https://open.spotify.com/show/7kyKbKwrqgxr4trONfkqmH?si=c4e17a6fc42f4a54)
- LinkedIn: https://www.linkedin.com/company/evoacmd/
- TikTok: https://www.tiktok.com/@evoacdm
- Instagram: https://www.instagram.com/evoacdm/
- YouTube: https://www.youtube.com/@evoacdm
- Sitio web: https://evoacademy.cl